In [1]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import inspect
import io 
import json
import numpy as np
import pandas as pd
import requests 
from scipy.optimize import curve_fit


In [2]:
urlData = requests.get('https://corona.lmao.ninja/v2/historical/argentina').content

In [3]:
data = urlData.decode('utf-8')
data

'{"country":"Argentina","province":["mainland"],"timeline":{"cases":{"7/23/20":148027,"7/24/20":153520,"7/25/20":158334,"7/26/20":162526,"7/27/20":167416,"7/28/20":173355,"7/29/20":178996,"7/30/20":185373,"7/31/20":191302,"8/1/20":196543,"8/2/20":201919,"8/3/20":206743,"8/4/20":213535,"8/5/20":220682,"8/6/20":228195,"8/7/20":235677,"8/8/20":241811,"8/9/20":246499,"8/10/20":253868,"8/11/20":260911,"8/12/20":268574,"8/13/20":276072,"8/14/20":282437,"8/15/20":289100,"8/16/20":294569,"8/17/20":299126,"8/18/20":305966,"8/19/20":312659,"8/20/20":320884,"8/21/20":329043},"deaths":{"7/23/20":2702,"7/24/20":2807,"7/25/20":2893,"7/26/20":2939,"7/27/20":3059,"7/28/20":3179,"7/29/20":3288,"7/30/20":3441,"7/31/20":3543,"8/1/20":3596,"8/2/20":3648,"8/3/20":3813,"8/4/20":3979,"8/5/20":4106,"8/6/20":4251,"8/7/20":4411,"8/8/20":4523,"8/9/20":4606,"8/10/20":4764,"8/11/20":5004,"8/12/20":5213,"8/13/20":5362,"8/14/20":5527,"8/15/20":5637,"8/16/20":5703,"8/17/20":5814,"8/18/20":6048,"8/19/20":6330,"8/20/20

In [4]:
data_load = json.loads(data)
data_load

{'country': 'Argentina',
 'province': ['mainland'],
 'timeline': {'cases': {'7/23/20': 148027,
   '7/24/20': 153520,
   '7/25/20': 158334,
   '7/26/20': 162526,
   '7/27/20': 167416,
   '7/28/20': 173355,
   '7/29/20': 178996,
   '7/30/20': 185373,
   '7/31/20': 191302,
   '8/1/20': 196543,
   '8/2/20': 201919,
   '8/3/20': 206743,
   '8/4/20': 213535,
   '8/5/20': 220682,
   '8/6/20': 228195,
   '8/7/20': 235677,
   '8/8/20': 241811,
   '8/9/20': 246499,
   '8/10/20': 253868,
   '8/11/20': 260911,
   '8/12/20': 268574,
   '8/13/20': 276072,
   '8/14/20': 282437,
   '8/15/20': 289100,
   '8/16/20': 294569,
   '8/17/20': 299126,
   '8/18/20': 305966,
   '8/19/20': 312659,
   '8/20/20': 320884,
   '8/21/20': 329043},
  'deaths': {'7/23/20': 2702,
   '7/24/20': 2807,
   '7/25/20': 2893,
   '7/26/20': 2939,
   '7/27/20': 3059,
   '7/28/20': 3179,
   '7/29/20': 3288,
   '7/30/20': 3441,
   '7/31/20': 3543,
   '8/1/20': 3596,
   '8/2/20': 3648,
   '8/3/20': 3813,
   '8/4/20': 3979,
   '8/5/2

In [5]:
df_timeline = pd.json_normalize(data_load['timeline']['cases']).T
df_deaths = pd.json_normalize(data_load['timeline']['deaths']).T
df_recovered = pd.json_normalize(data_load['timeline']['recovered']).T

In [6]:
df_timeline.columns = ['confirmed']
df_deaths.columns = ['deaths']
df_recovered.columns = ['recovered']

In [7]:
data_argentina = pd.concat([df_timeline, df_deaths, df_recovered], axis=1, sort=False)
data_argentina

,confirmed,deaths,recovered
7/23/20,148027,2702,62815
7/24/20,153520,2807,65447
7/25/20,158334,2893,68022
7/26/20,162526,2939,70518
7/27/20,167416,3059,72575
7/28/20,173355,3179,75083
7/29/20,178996,3288,77855
7/30/20,185373,3441,80596
7/31/20,191302,3543,83780
8/1/20,196543,3596,86499


In [8]:
data_argentina.index=pd.to_datetime(data_argentina.index.astype(str), format='%m/%d/%y')

In [9]:
data_argentina['num_id'] = np.arange(len(data_argentina) + 1)[1:]

In [10]:
data_argentina

,confirmed,deaths,recovered,num_id
2020-07-23,148027,2702,62815,1
2020-07-24,153520,2807,65447,2
2020-07-25,158334,2893,68022,3
2020-07-26,162526,2939,70518,4
2020-07-27,167416,3059,72575,5
2020-07-28,173355,3179,75083,6
2020-07-29,178996,3288,77855,7
2020-07-30,185373,3441,80596,8
2020-07-31,191302,3543,83780,9
2020-08-01,196543,3596,86499,10


In [11]:
predicted_days = 7
firstPredictionDay = data_argentina.index[-1] + datetime.timedelta(days = 1)

In [12]:
class Models:    
    def cuadratic_model(x, a, b, c):
        return (a*x**2 + b*x +c)
    
    def cubic_model(x, a, b, c, d):
        return (a*x**3 + b*x**2 +c*x + d)
    
    def fourth_pow_model(x, a, b, c, d, e):
        return (a*x**4 + b*x**3 +c*x**2 + d*x + e)
    
    def fiveth_pow_model(x, a, b, c, d, e, f):
        return (a*x**5 + b*x**4 +c*x**3 + d*x**2 + e*x + f)
    
    def sixh_pow_model(x, a, b, c, d, e, f, g):
        return (a*x**6 + b*x**5 +c*x**4 + d*x**3 + e*x**2 + f*x + g)

In [13]:
def formatPrediction(firstPredictionDay, day, prediction):
    formatedDate = str(firstPredictionDay + datetime.timedelta(days = day))[0:10]
    formatedPrediction = str(int(prediction))
    return "|" + formatedDate + " | " + formatedPrediction + "|"

def printPrediction(model, popt, x, y):
    firstPredictionDayId = x.max() + 1
    print("--- Predictions ---")
    for day in range (0, predicted_days):
        predictionId = firstPredictionDayId + day
        predictionValue = model(predictionId, *popt)
        print(formatPrediction(firstPredictionDay, day, predictionValue))
    print("-------------------")

In [14]:
def printFuncionParameters(model, popt):
        print('\n', model.__name__)
        args = inspect.getfullargspec(model).args[1:]
        for arg, value in zip(args, popt):
            print( arg, " = ", value)

In [15]:
def predictData(model, x, y):
    popt, _ = curve_fit(model, x, y)
    printPrediction(model, popt, x, y)
    printFuncionParameters(model, popt)
    print()

In [16]:
def isModel(methodName):
    return not(methodName.startswith('__'))

for methodName in dir(Models):
    if(isModel(methodName)):
        print(methodName)
        predictData(getattr(Models, methodName), data_argentina.num_id, data_argentina.confirmed)

cuadratic_model
--- Predictions ---
|2020-08-22 | 336366|
|2020-08-23 | 343687|
|2020-08-24 | 351072|
|2020-08-25 | 358520|
|2020-08-26 | 366032|
|2020-08-27 | 373609|
|2020-08-28 | 381249|
-------------------

 cuadratic_model
a  =  31.93560136205187
b  =  5308.791461732355
c  =  141104.09530912328

cubic_model
--- Predictions ---
|2020-08-22 | 333223|
|2020-08-23 | 339326|
|2020-08-24 | 345304|
|2020-08-25 | 351144|
|2020-08-26 | 356834|
|2020-08-27 | 362364|
|2020-08-28 | 367721|
-------------------

 cubic_model
a  =  -1.9206578827517253
b  =  121.24619459892985
c  =  4183.093826023771
d  =  144247.8283892946

fiveth_pow_model
--- Predictions ---
|2020-08-22 | 336513|
|2020-08-23 | 345665|
|2020-08-24 | 355980|
|2020-08-25 | 367754|
|2020-08-26 | 381319|
|2020-08-27 | 397051|
|2020-08-28 | 415367|
-------------------

 fiveth_pow_model
a  =  0.018545162742210297
b  =  -1.4106031324307953
c  =  36.381752186992784
d  =  -322.14445425056414
e  =  6200.975628421964
f  =  141804.6896201